In [18]:
import sklearn
import ast
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
from sklearn.tree import plot_tree

#Linear model
from sklearn.linear_model import LinearRegression

#KNN
from sklearn.neighbors import KNeighborsRegressor

#RF
from sklearn.ensemble import RandomForestRegressor

#SVR
from sklearn.svm import SVR

#Metrics
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error

#Feature selection
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

#Lasso
from sklearn.linear_model import Lasso

#feature selection
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

#Ridge
from sklearn.linear_model import Ridge

#Gradient Boosting 
#lightgbm
import lightgbm as lgb
#catboost
import catboost as ctb
#xgboost
import xgboost as xgb
import matplotlib as mpl

#Neural network
import math
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler


In [19]:
data = pd.read_csv('final_data_without_prefs_and_with_HAMADA_factors.csv')
data_ret = pd.read_csv('data_ret.csv')
data_ret.drop(columns=['Unnamed: 0'], inplace=True)
#data_ret.set_index('Date', inplace=True)
data.set_index('Date', inplace = True)
macro_data = pd.read_csv('macro_data.csv')
macro_data.set_index('Date', inplace = True)

ml_results_best_params = pd.read_csv('ML_best_params_ever.csv')
ml_data_1 = pd.read_csv('MSE_result_ever.csv')
ml_data = pd.read_csv('R2_result_ever.csv')

ml_data.set_index('share', inplace=True)
ml_data_1.set_index('share', inplace=True)
ml_results_best_params.set_index('share', inplace=True)

# Подгрузим данные
df = pd.read_csv('russian_data_without_prefs.csv')

# Удалим все дни где дневная доходность всех бумаг равна нулю (праздники и выходные)
df.drop(df[df.filter(regex='return_').sum(axis = 1) == 0].index, axis=0, inplace=True)
df.set_index('Date', inplace = True)

# Lasso

In [24]:
lag_autolag_return_30 = 30
lag_autolag_return_60 = 60
lag_autolag_return_90 = 90
lag_30 = 30
lag_60 = 60
lag_90 = 90


Long_returns_lasso = []
Short_returns_lasso = []

Long_10_lasso = []
Short_10_lasso = []
Hold_lasso = []

gap_10 = 0.01

for name in data_ret.columns[2:]:

    df1 = data[[name, 'Hamada_beta_MRP_for_{}'.format(name[7:11]), 'risk_free_rate_based_on_g_curve',
                 'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
                 'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]),
                 'MOM']].dropna()
    df_autolag = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)})
    df_autolag = pd.DataFrame(df_autolag['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)].shift(lag_autolag_return_30).dropna())

    df_autolag_2 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)})
    df_autolag_2 = pd.DataFrame(df_autolag_2['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)].shift(lag_autolag_return_60).dropna())

    df_autolag_3 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)})
    df_autolag_3 = pd.DataFrame(df_autolag_3['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)].shift(lag_autolag_return_90).dropna())

    df2 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_30).dropna()



    df2.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
                          'MOM':'MOM_lag_{}_days'.format(lag_30)}, inplace = True)

    result = pd.concat([df1, df2], axis=1, join="inner")

    df3 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_60).dropna()

    df3.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
                          'MOM':'MOM_lag_{}_days'.format(lag_60)}, inplace = True)

    result = pd.concat([result, df3], axis=1, join="inner")

    df4 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_90).dropna()

    df4.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90), 
                          'MOM':'MOM_lag_{}_days'.format(lag_90)}, inplace = True)

    result = pd.concat([result, df4], axis=1, join="inner")

    result = pd.merge(result, macro_data, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_2, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_3, left_index=True, right_index=True)
    result['{}_Equity_premium'.format(name[7:14])] = result[name] - result['risk_free_rate_based_on_g_curve']

    share = '{}_Equity_premium'.format(name[7:14])

    dta = result.dropna()

    X_train = dta.iloc[:round(len(dta) - len(dta)/4),:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    X_test = dta.iloc[-round(len(dta)/4):,:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    y_train = dta.iloc[:round(len(dta) - len(dta)/4),:][share]
    y_test = dta.iloc[-round(len(dta)/4):,:][share]

    lasso = Lasso(alpha = ast.literal_eval(ml_results_best_params.loc[share,'LASSO_best_params'])['alpha'],
                  normalize=ast.literal_eval(ml_results_best_params.loc[share,'LASSO_best_params'])['normalize'],
                  tol=ast.literal_eval(ml_results_best_params.loc[share,'LASSO_best_params'])['tol'])

    lasso.fit(X_train, y_train)
    y_pred_1 = lasso.predict(X_train.iloc[-1:])

    # фактическая больше модельной
    if y_train[-1] -  y_pred_1 > gap_10:
        Short_10_lasso.append(name)
    # модельная больше фактической
    elif y_pred_1 - y_train[-1] > gap_10:
        Long_10_lasso.append(name)
    elif abs(y_pred_1 - y_train[-1]) < gap_10:
            Hold_lasso.append(name)
    else:
        continue

Recomends_lasso = pd.DataFrame({'Short':pd.Series(Short_10_lasso), 
                                  'Hold':pd.Series(Hold_lasso),
                                  'Long':pd.Series(Long_10_lasso)})

Recomends_lasso['Short'] = Recomends_lasso['Short'].astype(str).str[7:11]
Recomends_lasso['Hold'] = Recomends_lasso['Hold'].astype(str).str[7:11]
Recomends_lasso['Long'] = Recomends_lasso['Long'].astype(str).str[7:11]
Recomends_lasso.to_excel('Great_recom_flat_lasso.xlsx', sheet_name='recom')


# CAPM

In [26]:
lag_autolag_return_30 = 30
lag_autolag_return_60 = 60
lag_autolag_return_90 = 90
lag_30 = 30
lag_60 = 60
lag_90 = 90


Long_returns_capm = []
Short_returns_capm = []

Long_10_capm = []
Short_10_capm = []
Hold_capm = []

window = 29
gap_10 = 0.01

for name in data_ret.columns[2:]:

    df1 = data[[name, 'Hamada_beta_MRP_for_{}'.format(name[7:11]), 'risk_free_rate_based_on_g_curve',
                 'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
                 'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]),
                 'MOM']].dropna()
    df_autolag = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)})
    df_autolag = pd.DataFrame(df_autolag['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)].shift(lag_autolag_return_30).dropna())

    df_autolag_2 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)})
    df_autolag_2 = pd.DataFrame(df_autolag_2['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)].shift(lag_autolag_return_60).dropna())

    df_autolag_3 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)})
    df_autolag_3 = pd.DataFrame(df_autolag_3['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)].shift(lag_autolag_return_90).dropna())

    df2 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_30).dropna()



    df2.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
                          'MOM':'MOM_lag_{}_days'.format(lag_30)}, inplace = True)

    result = pd.concat([df1, df2], axis=1, join="inner")

    df3 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_60).dropna()

    df3.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
                          'MOM':'MOM_lag_{}_days'.format(lag_60)}, inplace = True)

    result = pd.concat([result, df3], axis=1, join="inner")

    df4 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_90).dropna()

    df4.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90), 
                          'MOM':'MOM_lag_{}_days'.format(lag_90)}, inplace = True)

    result = pd.concat([result, df4], axis=1, join="inner")

    result = pd.merge(result, macro_data, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_2, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_3, left_index=True, right_index=True)
    result['{}_Equity_premium'.format(name[7:14])] = result[name] - result['risk_free_rate_based_on_g_curve']

    share = '{}_Equity_premium'.format(name[7:14])

    dta = result.dropna()

    X_train = dta.iloc[:round(len(dta) - len(dta)/4),:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    X_test = dta.iloc[-round(len(dta)/4):,:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    y_train = dta.iloc[:round(len(dta) - len(dta)/4),:][share]
    y_test = dta.iloc[-round(len(dta)/4):,:][share]

    model = LinearRegression(fit_intercept=False)
    model.fit(X_train.iloc[:,0:1], y_train)

    y_pred_1 = model.predict(X_train.iloc[-1:,0:1])

    # фактическая больше модельной
    if y_train[-1] -  y_pred_1 > gap_10:
        Short_10_capm.append(name)
    # модельная больше фактической
    elif y_pred_1 - y_train[-1] > gap_10:
        Long_10_capm.append(name)
    elif abs(y_pred_1 - y_train[-1]) < gap_10:
        Hold_capm.append(name)
    else:
        continue




Recomends_capm = pd.DataFrame({'Short':pd.Series(Short_10_capm), 
                                  'Hold':pd.Series(Hold_capm),
                                  'Long':pd.Series(Long_10_capm)})

Recomends_capm['Short'] = Recomends_capm['Short'].astype(str).str[7:11]
Recomends_capm['Hold'] = Recomends_capm['Hold'].astype(str).str[7:11]
Recomends_capm['Long'] = Recomends_capm['Long'].astype(str).str[7:11]
Recomends_capm.to_excel('Great_recom_flat_capm.xlsx', sheet_name='recom')


       


# Fama-French

In [31]:
lag_autolag_return_30 = 30
lag_autolag_return_60 = 60
lag_autolag_return_90 = 90
lag_30 = 30
lag_60 = 60
lag_90 = 90


Long_returns_ff = []
Short_returns_ff = []

Long_10_ff = []
Short_10_ff = []
Hold_ff = []

gap_10 = 0.01

for name in data_ret.columns[2:4]:

    df1 = data[[name, 'Hamada_beta_MRP_for_{}'.format(name[7:11]), 'risk_free_rate_based_on_g_curve',
                 'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
                 'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]),
                 'MOM']].dropna()
    df_autolag = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)})
    df_autolag = pd.DataFrame(df_autolag['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)].shift(lag_autolag_return_30).dropna())

    df_autolag_2 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)})
    df_autolag_2 = pd.DataFrame(df_autolag_2['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)].shift(lag_autolag_return_60).dropna())

    df_autolag_3 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)})
    df_autolag_3 = pd.DataFrame(df_autolag_3['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)].shift(lag_autolag_return_90).dropna())

    df2 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_30).dropna()



    df2.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
                          'MOM':'MOM_lag_{}_days'.format(lag_30)}, inplace = True)

    result = pd.concat([df1, df2], axis=1, join="inner")

    df3 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_60).dropna()

    df3.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
                          'MOM':'MOM_lag_{}_days'.format(lag_60)}, inplace = True)

    result = pd.concat([result, df3], axis=1, join="inner")

    df4 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_90).dropna()

    df4.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90), 
                          'MOM':'MOM_lag_{}_days'.format(lag_90)}, inplace = True)

    result = pd.concat([result, df4], axis=1, join="inner")

    result = pd.merge(result, macro_data, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_2, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_3, left_index=True, right_index=True)
    result['{}_Equity_premium'.format(name[7:14])] = result[name] - result['risk_free_rate_based_on_g_curve']

    share = '{}_Equity_premium'.format(name[7:14])

    dta = result.dropna()

    X_train = dta.iloc[:round(len(dta) - len(dta)/4),:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    X_test = dta.iloc[-round(len(dta)/4):,:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    y_train = dta.iloc[:round(len(dta) - len(dta)/4),:][share]
    y_test = dta.iloc[-round(len(dta)/4):,:][share]

    model = LinearRegression(fit_intercept=False)
    model.fit(X_train.iloc[:,[0, 2, 3]], y_train)

    y_pred_1 = model.predict(X_train.iloc[-1:,[0, 2, 3]])

    # фактическая больше модельной
    if y_train[-1] -  y_pred_1 > gap_10:
        Short_10_ff.append(name)
    # модельная больше фактической
    elif y_pred_1 - y_train[-1] > gap_10:
        Long_10_ff.append(name)
    elif abs(y_pred_1 - y_train[-1]) < gap_10:
        Hold_ff.append(name)
    else:
        continue


        
Recomends_FF = pd.DataFrame({'Short':pd.Series(Short_10_ff), 
                                  'Hold':pd.Series(Hold_ff),
                                  'Long':pd.Series(Long_10_ff)})

Recomends_FF['Short'] = Recomends_FF['Short'].astype(str).str[7:11]
Recomends_FF['Hold'] = Recomends_FF['Hold'].astype(str).str[7:11]
Recomends_FF['Long'] = Recomends_FF['Long'].astype(str).str[7:11]

Recomends_FF.to_excel('Great_recom_flat_ff.xlsx', sheet_name='Рекомы_Фама_Френч')



<ipython-input-31-1b5b8c012566>:124: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  'Hold':pd.Series(Hold_ff),


# Make data

In [32]:
my_df = df.loc['2018-04-24':'2018-09-01', 'PX_LAST_IMOEX_Index':'PX_LAST_VZRZ_RM_Equity']



In [33]:
my_df

,PX_LAST_IMOEX_Index,PX_TO_BOOK_RATIO_IMOEX_Index,CURRENT_MARKET_CAP_SHARE_CLASS_IMOEX_Index,DEBT_TO_MKT_CAP_IMOEX_Index,return_IMOEX_Index,Dates_SBER_RM_Equity,PX_LAST_SBER_RM_Equity,PX_TO_BOOK_RATIO_SBER_RM_Equity,CURRENT_MARKET_CAP_SHARE_CLASS_SBER_RM_Equity,DEBT_TO_MKT_CAP_SBER_RM_Equity,...,DEBT_TO_MKT_CAP_RASP_RM_Equity,return_RASP_RM_Equity,Dates_OGKB_RM_Equity,PX_LAST_OGKB_RM_Equity,PX_TO_BOOK_RATIO_OGKB_RM_Equity,CURRENT_MARKET_CAP_SHARE_CLASS_OGKB_RM_Equity,DEBT_TO_MKT_CAP_OGKB_RM_Equity,return_OGKB_RM_Equity,Dates_VZRZ_RM_Equity,PX_LAST_VZRZ_RM_Equity
Date,,,,,,,,,,,,,,,,,,,,,
2018-04-24,2253.77,0.7654,NaN,NaN,-0.028696,2018-04-24,223.00,1.3467,4.813889e+06,0.5372,...,0.0359,-0.212545,2018-04-24,0.4500,0.3797,49698.5224,1.1550,-0.054067,2018-04-24,335.5
2018-04-25,2253.50,0.7653,NaN,NaN,-0.009601,2018-04-25,220.45,1.3313,4.758843e+06,0.5434,...,0.0367,-0.197505,2018-04-25,0.4474,0.3775,49411.3754,1.1617,-0.048641,2018-04-25,328.5
2018-04-26,2286.02,0.7764,NaN,NaN,0.005936,2018-04-26,220.70,1.3328,4.764239e+06,0.5428,...,0.0353,-0.147624,2018-04-26,0.4416,0.3726,48770.8166,1.1770,-0.050988,2018-04-26,322.5
2018-04-27,2301.49,0.7816,NaN,NaN,0.002998,2018-04-27,223.91,1.3522,4.833534e+06,0.5350,...,0.0344,-0.142055,2018-04-27,0.4540,0.3831,50140.2870,1.1448,-0.012912,2018-04-27,326.5
2018-04-30,2307.02,0.7835,NaN,NaN,0.011027,2018-04-30,226.99,1.3708,4.900021e+06,0.5278,...,0.0350,-0.166943,2018-04-30,0.4539,0.3830,50129.2429,1.1451,-0.012480,2018-04-30,336.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-27,2312.93,0.8013,NaN,NaN,-0.009146,2018-08-27,183.27,1.1368,3.956240e+06,0.5574,...,0.0000,-0.040232,2018-08-27,0.3460,0.2925,38212.6417,1.2714,-0.098982,2018-08-27,412.0
2018-08-28,2316.28,0.8024,NaN,NaN,0.002317,2018-08-28,182.00,1.1289,3.928825e+06,0.5613,...,0.0000,-0.050363,2018-08-28,0.3431,0.2900,37892.3623,1.2821,-0.117297,2018-08-28,407.0
2018-08-29,2348.44,0.8136,NaN,NaN,0.023114,2018-08-29,183.50,1.1382,3.961205e+06,0.5567,...,0.0000,-0.037453,2018-08-29,0.3450,0.2916,38102.2005,1.2751,-0.100304,2018-08-29,409.0


In [34]:
my_df = my_df.filter(regex = 'PX_LAST')

In [35]:
my_df.columns = my_df.columns.astype(str).str[8:12]

In [36]:
my_df.rename(columns={'IMOE':'IMOEX'},inplace=True)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [37]:
my_df.to_excel('FLAT_data.xlsx', sheet_name='Цены')

# Ridge

In [359]:
lag_autolag_return_30 = 30
lag_autolag_return_60 = 60
lag_autolag_return_90 = 90
lag_30 = 30
lag_60 = 60
lag_90 = 90


Long_returns_ridge = []
Short_returns_ridge = []

Long_10_ridge = []
Short_10_ridge = []
Hold_ridge = []

gap_10 = 0.01

for name in data_ret.columns[2:]:

    df1 = data[[name, 'Hamada_beta_MRP_for_{}'.format(name[7:11]), 'risk_free_rate_based_on_g_curve',
                 'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
                 'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]),
                 'MOM']].dropna()
    df_autolag = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)})
    df_autolag = pd.DataFrame(df_autolag['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)].shift(lag_autolag_return_30).dropna())

    df_autolag_2 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)})
    df_autolag_2 = pd.DataFrame(df_autolag_2['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)].shift(lag_autolag_return_60).dropna())

    df_autolag_3 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)})
    df_autolag_3 = pd.DataFrame(df_autolag_3['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)].shift(lag_autolag_return_90).dropna())

    df2 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_30).dropna()



    df2.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
                          'MOM':'MOM_lag_{}_days'.format(lag_30)}, inplace = True)

    result = pd.concat([df1, df2], axis=1, join="inner")

    df3 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_60).dropna()

    df3.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
                          'MOM':'MOM_lag_{}_days'.format(lag_60)}, inplace = True)

    result = pd.concat([result, df3], axis=1, join="inner")

    df4 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_90).dropna()

    df4.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90), 
                          'MOM':'MOM_lag_{}_days'.format(lag_90)}, inplace = True)

    result = pd.concat([result, df4], axis=1, join="inner")

    result = pd.merge(result, macro_data, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_2, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_3, left_index=True, right_index=True)
    result['{}_Equity_premium'.format(name[7:14])] = result[name] - result['risk_free_rate_based_on_g_curve']

    share = '{}_Equity_premium'.format(name[7:14])

    dta = result.dropna()

    X_train = dta.iloc[:round(len(dta) - len(dta)/4),:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    X_test = dta.iloc[-round(len(dta)/4):,:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    y_train = dta.iloc[:round(len(dta) - len(dta)/4),:][share]
    y_test = dta.iloc[-round(len(dta)/4):,:][share]

    ridge = Ridge(alpha=ast.literal_eval(ml_results_best_params.loc[share,'Ridge_best_params'])['alpha'],
                  tol=ast.literal_eval(ml_results_best_params.loc[share,'Ridge_best_params'])['tol'],
                  normalize=ast.literal_eval(ml_results_best_params.loc[share,'Ridge_best_params'])['normalize'])

    ridge.fit(X_train, y_train)
    y_pred_1 = ridge.predict(X_train.iloc[-1:])

    # фактическая больше модельной
    if y_train[-1] -  y_pred_1 > gap_10:
        Short_10_ridge.append(name)
    # модельная больше фактической
    elif y_pred_1 - y_train[-1] > gap_10:
        Long_10_ridge.append(name)
    elif abs(y_pred_1 - y_train[-1]) < gap_10:
            Hold_ridge.append(name)
    else:
        continue

In [318]:
Recomends_ridge = pd.DataFrame({'Short':pd.Series(Short_10_ridge), 
                                  'Hold':pd.Series(Hold_ridge),
                                  'Long':pd.Series(Long_10_ridge)})

Recomends_ridge['Short'] = Recomends_ridge['Short'].astype(str).str[7:11]
Recomends_ridge['Hold'] = Recomends_ridge['Hold'].astype(str).str[7:11]
Recomends_ridge['Long'] = Recomends_ridge['Long'].astype(str).str[7:11]
Recomends_ridge.to_excel('Recomends_data_ridge.xlsx', sheet_name='recom')

# SVM

In [322]:
lag_autolag_return_30 = 30
lag_autolag_return_60 = 60
lag_autolag_return_90 = 90
lag_30 = 30
lag_60 = 60
lag_90 = 90


Long_10_svm = []
Short_10_svm = []
Hold_svm = []

gap_10 = 0.01

for name in data_ret.columns[2:]:

    df1 = data[[name, 'Hamada_beta_MRP_for_{}'.format(name[7:11]), 'risk_free_rate_based_on_g_curve',
                 'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
                 'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]),
                 'MOM']].dropna()
    df_autolag = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)})
    df_autolag = pd.DataFrame(df_autolag['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)].shift(lag_autolag_return_30).dropna())

    df_autolag_2 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)})
    df_autolag_2 = pd.DataFrame(df_autolag_2['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)].shift(lag_autolag_return_60).dropna())

    df_autolag_3 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)})
    df_autolag_3 = pd.DataFrame(df_autolag_3['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)].shift(lag_autolag_return_90).dropna())

    df2 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_30).dropna()



    df2.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
                          'MOM':'MOM_lag_{}_days'.format(lag_30)}, inplace = True)

    result = pd.concat([df1, df2], axis=1, join="inner")

    df3 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_60).dropna()

    df3.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
                          'MOM':'MOM_lag_{}_days'.format(lag_60)}, inplace = True)

    result = pd.concat([result, df3], axis=1, join="inner")

    df4 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_90).dropna()

    df4.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90), 
                          'MOM':'MOM_lag_{}_days'.format(lag_90)}, inplace = True)

    result = pd.concat([result, df4], axis=1, join="inner")

    result = pd.merge(result, macro_data, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_2, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_3, left_index=True, right_index=True)
    result['{}_Equity_premium'.format(name[7:14])] = result[name] - result['risk_free_rate_based_on_g_curve']

    share = '{}_Equity_premium'.format(name[7:14])

    dta = result.dropna()

    X_train = dta.iloc[:round(len(dta) - len(dta)/4),:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    X_test = dta.iloc[-round(len(dta)/4):,:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    y_train = dta.iloc[:round(len(dta) - len(dta)/4),:][share]
    y_test = dta.iloc[-round(len(dta)/4):,:][share]

    svm = SVR(C = ast.literal_eval(ml_results_best_params.loc[share,'SVM_best_params'])['C'],
            gamma = ast.literal_eval(ml_results_best_params.loc[share,'SVM_best_params'])['gamma'],
            kernel = ast.literal_eval(ml_results_best_params.loc[share,'SVM_best_params'])['kernel'])

    svm.fit(X_train, y_train)
    y_pred_1 = svm.predict(X_train.iloc[-1:])

    # фактическая больше модельной
    if y_train[-1] -  y_pred_1 > gap_10:
        Short_10_svm.append(name)
    # модельная больше фактической
    elif y_pred_1 - y_train[-1] > gap_10:
        Long_10_svm.append(name)
    elif abs(y_pred_1 - y_train[-1]) < gap_10:
            Hold_svm.append(name)
    else:
        continue
        
Recomends_svm = pd.DataFrame({'Short':pd.Series(Short_10_svm), 
                                  'Hold':pd.Series(Hold_svm),
                                  'Long':pd.Series(Long_10_svm)})

Recomends_svm['Short'] = Recomends_svm['Short'].astype(str).str[7:11]
Recomends_svm['Hold'] = Recomends_svm['Hold'].astype(str).str[7:11]
Recomends_svm['Long'] = Recomends_svm['Long'].astype(str).str[7:11]
Recomends_svm.to_excel('Recomends_data_svm.xlsx', sheet_name='recom')        


# RF

In [324]:
lag_autolag_return_30 = 30
lag_autolag_return_60 = 60
lag_autolag_return_90 = 90
lag_30 = 30
lag_60 = 60
lag_90 = 90


Long_10_rf = []
Short_10_rf = []
Hold_rf = []

gap_10 = 0.01

for name in data_ret.columns[2:]:

    df1 = data[[name, 'Hamada_beta_MRP_for_{}'.format(name[7:11]), 'risk_free_rate_based_on_g_curve',
                 'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
                 'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]),
                 'MOM']].dropna()
    df_autolag = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)})
    df_autolag = pd.DataFrame(df_autolag['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)].shift(lag_autolag_return_30).dropna())

    df_autolag_2 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)})
    df_autolag_2 = pd.DataFrame(df_autolag_2['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)].shift(lag_autolag_return_60).dropna())

    df_autolag_3 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)})
    df_autolag_3 = pd.DataFrame(df_autolag_3['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)].shift(lag_autolag_return_90).dropna())

    df2 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_30).dropna()



    df2.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
                          'MOM':'MOM_lag_{}_days'.format(lag_30)}, inplace = True)

    result = pd.concat([df1, df2], axis=1, join="inner")

    df3 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_60).dropna()

    df3.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
                          'MOM':'MOM_lag_{}_days'.format(lag_60)}, inplace = True)

    result = pd.concat([result, df3], axis=1, join="inner")

    df4 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_90).dropna()

    df4.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90), 
                          'MOM':'MOM_lag_{}_days'.format(lag_90)}, inplace = True)

    result = pd.concat([result, df4], axis=1, join="inner")

    result = pd.merge(result, macro_data, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_2, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_3, left_index=True, right_index=True)
    result['{}_Equity_premium'.format(name[7:14])] = result[name] - result['risk_free_rate_based_on_g_curve']

    share = '{}_Equity_premium'.format(name[7:14])

    dta = result.dropna()

    X_train = dta.iloc[:round(len(dta) - len(dta)/4),:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    X_test = dta.iloc[-round(len(dta)/4):,:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    y_train = dta.iloc[:round(len(dta) - len(dta)/4),:][share]
    y_test = dta.iloc[-round(len(dta)/4):,:][share]

    rf = RandomForestRegressor(n_estimators = ast.literal_eval(ml_results_best_params.loc[share,'RF_best_params'])['n_estimators'],
                      max_features=ast.literal_eval(ml_results_best_params.loc[share,'RF_best_params'])['max_features'],
                      min_samples_leaf=ast.literal_eval(ml_results_best_params.loc[share,'RF_best_params'])['min_samples_leaf'],
                      min_samples_split=ast.literal_eval(ml_results_best_params.loc[share,'RF_best_params'])['min_samples_split'],
                      max_depth=ast.literal_eval(ml_results_best_params.loc[share,'RF_best_params'])['max_depth'],
                      bootstrap = ast.literal_eval(ml_results_best_params.loc[share,'RF_best_params'])['bootstrap'])
    rf.fit(X_train, y_train)
    y_pred_1 = rf.predict(X_train.iloc[-1:])

    # фактическая больше модельной
    if y_train[-1] -  y_pred_1 > gap_10:
        Short_10_rf.append(name)
    # модельная больше фактической
    elif y_pred_1 - y_train[-1] > gap_10:
        Long_10_rf.append(name)
    elif abs(y_pred_1 - y_train[-1]) < gap_10:
            Hold_rf.append(name)
    else:
        continue
        
Recomends_rf = pd.DataFrame({'Short':pd.Series(Short_10_rf), 
                                  'Hold':pd.Series(Hold_rf),
                                  'Long':pd.Series(Long_10_rf)})

Recomends_rf['Short'] = Recomends_rf['Short'].astype(str).str[7:11]
Recomends_rf['Hold'] = Recomends_rf['Hold'].astype(str).str[7:11]
Recomends_rf['Long'] = Recomends_rf['Long'].astype(str).str[7:11]
Recomends_rf.to_excel('Recomends_data_rf.xlsx', sheet_name='recom')        


# XGBoost

In [325]:
lag_autolag_return_30 = 30
lag_autolag_return_60 = 60
lag_autolag_return_90 = 90
lag_30 = 30
lag_60 = 60
lag_90 = 90


Long_10_xgb = []
Short_10_xgb = []
Hold_xgb = []

gap_10 = 0.01

for name in data_ret.columns[2:]:

    df1 = data[[name, 'Hamada_beta_MRP_for_{}'.format(name[7:11]), 'risk_free_rate_based_on_g_curve',
                 'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
                 'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]),
                 'MOM']].dropna()
    df_autolag = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)})
    df_autolag = pd.DataFrame(df_autolag['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)].shift(lag_autolag_return_30).dropna())

    df_autolag_2 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)})
    df_autolag_2 = pd.DataFrame(df_autolag_2['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)].shift(lag_autolag_return_60).dropna())

    df_autolag_3 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)})
    df_autolag_3 = pd.DataFrame(df_autolag_3['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)].shift(lag_autolag_return_90).dropna())

    df2 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_30).dropna()



    df2.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
                          'MOM':'MOM_lag_{}_days'.format(lag_30)}, inplace = True)

    result = pd.concat([df1, df2], axis=1, join="inner")

    df3 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_60).dropna()

    df3.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
                          'MOM':'MOM_lag_{}_days'.format(lag_60)}, inplace = True)

    result = pd.concat([result, df3], axis=1, join="inner")

    df4 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_90).dropna()

    df4.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90), 
                          'MOM':'MOM_lag_{}_days'.format(lag_90)}, inplace = True)

    result = pd.concat([result, df4], axis=1, join="inner")

    result = pd.merge(result, macro_data, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_2, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_3, left_index=True, right_index=True)
    result['{}_Equity_premium'.format(name[7:14])] = result[name] - result['risk_free_rate_based_on_g_curve']

    share = '{}_Equity_premium'.format(name[7:14])

    dta = result.dropna()

    X_train = dta.iloc[:round(len(dta) - len(dta)/4),:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    X_test = dta.iloc[-round(len(dta)/4):,:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    y_train = dta.iloc[:round(len(dta) - len(dta)/4),:][share]
    y_test = dta.iloc[-round(len(dta)/4):,:][share]

    model = xgb.XGBRegressor(n_estimators = ast.literal_eval(ml_results_best_params.loc[share,'XGBoost_best_params'])['n_estimators'],
                     colsample_bytree = ast.literal_eval(ml_results_best_params.loc[share,'XGBoost_best_params'])['colsample_bytree'],
                     learning_rate = ast.literal_eval(ml_results_best_params.loc[share,'XGBoost_best_params'])['learning_rate'],
                     max_depth = ast.literal_eval(ml_results_best_params.loc[share,'XGBoost_best_params'])['max_depth'],
                     min_child_weight = ast.literal_eval(ml_results_best_params.loc[share,'XGBoost_best_params'])['min_child_weight'],
                     random_state = ast.literal_eval(ml_results_best_params.loc[share,'XGBoost_best_params'])['random_state'],
                     reg_alpha = ast.literal_eval(ml_results_best_params.loc[share,'XGBoost_best_params'])['reg_alpha'],
                     reg_lambda = ast.literal_eval(ml_results_best_params.loc[share,'XGBoost_best_params'])['reg_lambda'],
                     subsample = ast.literal_eval(ml_results_best_params.loc[share,'XGBoost_best_params'])['subsample'])


    model.fit(X_train, y_train)
    
    y_pred_1 = model.predict(X_train.iloc[-1:])

    # фактическая больше модельной
    if y_train[-1] -  y_pred_1 > gap_10:
        Short_10_xgb.append(name)
    # модельная больше фактической
    elif y_pred_1 - y_train[-1] > gap_10:
        Long_10_xgb.append(name)
    elif abs(y_pred_1 - y_train[-1]) < gap_10:
            Hold_xgb.append(name)
    else:
        continue
        
Recomends_xgb = pd.DataFrame({'Short':pd.Series(Short_10_rf), 
                                  'Hold':pd.Series(Hold_rf),
                                  'Long':pd.Series(Long_10_rf)})

Recomends_xgb['Short'] = Recomends_xgb['Short'].astype(str).str[7:11]
Recomends_xgb['Hold'] = Recomends_xgb['Hold'].astype(str).str[7:11]
Recomends_xgb['Long'] = Recomends_xgb['Long'].astype(str).str[7:11]
Recomends_xgb.to_excel('Recomends_data_xgb.xlsx', sheet_name='recom')        


# Catboost

In [339]:
lag_autolag_return_30 = 30
lag_autolag_return_60 = 60
lag_autolag_return_90 = 90
lag_30 = 30
lag_60 = 60
lag_90 = 90


Long_10_ctb = []
Short_10_ctb = []
Hold_ctb = []

gap_10 = 0.01

for name in data_ret.columns[2:4]:

    df1 = data[[name, 'Hamada_beta_MRP_for_{}'.format(name[7:11]), 'risk_free_rate_based_on_g_curve',
                 'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
                 'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]),
                 'MOM']].dropna()
    df_autolag = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)})
    df_autolag = pd.DataFrame(df_autolag['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)].shift(lag_autolag_return_30).dropna())

    df_autolag_2 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)})
    df_autolag_2 = pd.DataFrame(df_autolag_2['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)].shift(lag_autolag_return_60).dropna())

    df_autolag_3 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)})
    df_autolag_3 = pd.DataFrame(df_autolag_3['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)].shift(lag_autolag_return_90).dropna())

    df2 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_30).dropna()



    df2.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
                          'MOM':'MOM_lag_{}_days'.format(lag_30)}, inplace = True)

    result = pd.concat([df1, df2], axis=1, join="inner")

    df3 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_60).dropna()

    df3.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
                          'MOM':'MOM_lag_{}_days'.format(lag_60)}, inplace = True)

    result = pd.concat([result, df3], axis=1, join="inner")

    df4 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_90).dropna()

    df4.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90), 
                          'MOM':'MOM_lag_{}_days'.format(lag_90)}, inplace = True)

    result = pd.concat([result, df4], axis=1, join="inner")

    result = pd.merge(result, macro_data, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_2, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_3, left_index=True, right_index=True)
    result['{}_Equity_premium'.format(name[7:14])] = result[name] - result['risk_free_rate_based_on_g_curve']

    share = '{}_Equity_premium'.format(name[7:14])

    dta = result.dropna()

    X_train = dta.iloc[:round(len(dta) - len(dta)/4),:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    X_test = dta.iloc[-round(len(dta)/4):,:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    y_train = dta.iloc[:round(len(dta) - len(dta)/4),:][share]
    y_test = dta.iloc[-round(len(dta)/4):,:][share]

    model = ctb.CatBoostRegressor(depth = ast.literal_eval(ml_results_best_params.loc[share,'Catboost_best_params'])['depth'],
                     iterations = ast.literal_eval(ml_results_best_params.loc[share,'Catboost_best_params'])['iterations'],
                     learning_rate = ast.literal_eval(ml_results_best_params.loc[share,'Catboost_best_params'])['learning_rate'],
                     l2_leaf_reg = ast.literal_eval(ml_results_best_params.loc[share,'Catboost_best_params'])['l2_leaf_reg'],
                     silent = ast.literal_eval(ml_results_best_params.loc[share,'Catboost_best_params'])['silent'])


    model.fit(X_train, y_train)
    
    y_pred_1 = model.predict(X_train.iloc[-1:])

    # фактическая больше модельной
    if y_train[-1] -  y_pred_1 > gap_10:
        Short_10_ctb.append(name)
    # модельная больше фактической
    elif y_pred_1 - y_train[-1] > gap_10:
        Long_10_ctb.append(name)
    elif abs(y_pred_1 - y_train[-1]) < gap_10:
            Hold_ctb.append(name)
    else:
        continue
        
Recomends_ctb = pd.DataFrame({'Short':pd.Series(Short_10_rf), 
                                  'Hold':pd.Series(Hold_rf),
                                  'Long':pd.Series(Long_10_rf)})

Recomends_ctb['Short'] = Recomends_ctb['Short'].astype(str).str[7:11]
Recomends_ctb['Hold'] = Recomends_ctb['Hold'].astype(str).str[7:11]
Recomends_ctb['Long'] = Recomends_ctb['Long'].astype(str).str[7:11]
#Recomends_ctb.to_excel('Recomends_data_ctb.xlsx', sheet_name='recom')        


# KNN

In [329]:
lag_autolag_return_30 = 30
lag_autolag_return_60 = 60
lag_autolag_return_90 = 90
lag_30 = 30
lag_60 = 60
lag_90 = 90


Long_10_knn = []
Short_10_knn = []
Hold_knn = []

gap_10 = 0.01

for name in data_ret.columns[2:]:

    df1 = data[[name, 'Hamada_beta_MRP_for_{}'.format(name[7:11]), 'risk_free_rate_based_on_g_curve',
                 'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
                 'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]),
                 'MOM']].dropna()
    df_autolag = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)})
    df_autolag = pd.DataFrame(df_autolag['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)].shift(lag_autolag_return_30).dropna())

    df_autolag_2 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)})
    df_autolag_2 = pd.DataFrame(df_autolag_2['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)].shift(lag_autolag_return_60).dropna())

    df_autolag_3 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)})
    df_autolag_3 = pd.DataFrame(df_autolag_3['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)].shift(lag_autolag_return_90).dropna())

    df2 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_30).dropna()



    df2.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
                          'MOM':'MOM_lag_{}_days'.format(lag_30)}, inplace = True)

    result = pd.concat([df1, df2], axis=1, join="inner")

    df3 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_60).dropna()

    df3.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
                          'MOM':'MOM_lag_{}_days'.format(lag_60)}, inplace = True)

    result = pd.concat([result, df3], axis=1, join="inner")

    df4 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_90).dropna()

    df4.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90), 
                          'MOM':'MOM_lag_{}_days'.format(lag_90)}, inplace = True)

    result = pd.concat([result, df4], axis=1, join="inner")

    result = pd.merge(result, macro_data, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_2, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_3, left_index=True, right_index=True)
    result['{}_Equity_premium'.format(name[7:14])] = result[name] - result['risk_free_rate_based_on_g_curve']

    share = '{}_Equity_premium'.format(name[7:14])

    dta = result.dropna()

    X_train = dta.iloc[:round(len(dta) - len(dta)/4),:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    X_test = dta.iloc[-round(len(dta)/4):,:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    y_train = dta.iloc[:round(len(dta) - len(dta)/4),:][share]
    y_test = dta.iloc[-round(len(dta)/4):,:][share]

    model = KNeighborsRegressor(n_neighbors=ast.literal_eval(ml_results_best_params.loc[share,'KNN_best_params'])['n_neighbors'],
                    weights = ast.literal_eval(ml_results_best_params.loc[share,'KNN_best_params'])['weights'],
                    p=ast.literal_eval(ml_results_best_params.loc[share,'KNN_best_params'])['p'],
                    algorithm=ast.literal_eval(ml_results_best_params.loc[share,'KNN_best_params'])['algorithm'])
    model.fit(X_train, y_train)
    
    y_pred_1 = model.predict(X_train.iloc[-1:])

    # фактическая больше модельной
    if y_train[-1] -  y_pred_1 > gap_10:
        Short_10_knn.append(name)
    # модельная больше фактической
    elif y_pred_1 - y_train[-1] > gap_10:
        Long_10_knn.append(name)
    elif abs(y_pred_1 - y_train[-1]) < gap_10:
            Hold_knn.append(name)
    else:
        continue
        
Recomends_knn = pd.DataFrame({'Short':pd.Series(Short_10_rf), 
                                  'Hold':pd.Series(Hold_rf),
                                  'Long':pd.Series(Long_10_rf)})

Recomends_knn['Short'] = Recomends_knn['Short'].astype(str).str[7:11]
Recomends_knn['Hold'] = Recomends_knn['Hold'].astype(str).str[7:11]
Recomends_knn['Long'] = Recomends_knn['Long'].astype(str).str[7:11]
Recomends_knn.to_excel('Recomends_data_knn.xlsx', sheet_name='recom')        


# Ansemble

In [331]:
lag_autolag_return_30 = 30
lag_autolag_return_60 = 60
lag_autolag_return_90 = 90
lag_30 = 30
lag_60 = 60
lag_90 = 90


Long_10_all = []
Short_10_all = []
Hold_all = []

gap_10 = 0.01

for name in data_ret.columns[2:]:

    df1 = data[[name, 'Hamada_beta_MRP_for_{}'.format(name[7:11]), 'risk_free_rate_based_on_g_curve',
                 'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
                 'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]),
                 'MOM']].dropna()
    df_autolag = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)})
    df_autolag = pd.DataFrame(df_autolag['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)].shift(lag_autolag_return_30).dropna())

    df_autolag_2 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)})
    df_autolag_2 = pd.DataFrame(df_autolag_2['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)].shift(lag_autolag_return_60).dropna())

    df_autolag_3 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)})
    df_autolag_3 = pd.DataFrame(df_autolag_3['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)].shift(lag_autolag_return_90).dropna())

    df2 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_30).dropna()



    df2.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
                          'MOM':'MOM_lag_{}_days'.format(lag_30)}, inplace = True)

    result = pd.concat([df1, df2], axis=1, join="inner")

    df3 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_60).dropna()

    df3.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
                          'MOM':'MOM_lag_{}_days'.format(lag_60)}, inplace = True)

    result = pd.concat([result, df3], axis=1, join="inner")

    df4 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_90).dropna()

    df4.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90), 
                          'MOM':'MOM_lag_{}_days'.format(lag_90)}, inplace = True)

    result = pd.concat([result, df4], axis=1, join="inner")

    result = pd.merge(result, macro_data, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_2, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_3, left_index=True, right_index=True)
    result['{}_Equity_premium'.format(name[7:14])] = result[name] - result['risk_free_rate_based_on_g_curve']

    share = '{}_Equity_premium'.format(name[7:14])

    dta = result.dropna()

    X_train = dta.iloc[:round(len(dta) - len(dta)/4),:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    X_test = dta.iloc[-round(len(dta)/4):,:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    y_train = dta.iloc[:round(len(dta) - len(dta)/4),:][share]
    y_test = dta.iloc[-round(len(dta)/4):,:][share]

    #knn
    model = KNeighborsRegressor(n_neighbors=ast.literal_eval(ml_results_best_params.loc[share,'KNN_best_params'])['n_neighbors'],
                    weights = ast.literal_eval(ml_results_best_params.loc[share,'KNN_best_params'])['weights'],
                    p=ast.literal_eval(ml_results_best_params.loc[share,'KNN_best_params'])['p'],
                    algorithm=ast.literal_eval(ml_results_best_params.loc[share,'KNN_best_params'])['algorithm'])
    model.fit(X_train, y_train)
    
    y_pred_1 = model.predict(X_train.iloc[-1:])

    
    #catboost
    model = ctb.CatBoostRegressor(depth = ast.literal_eval(ml_results_best_params.loc[share,'Catboost_best_params'])['depth'],
                     iterations = ast.literal_eval(ml_results_best_params.loc[share,'Catboost_best_params'])['iterations'],
                     learning_rate = ast.literal_eval(ml_results_best_params.loc[share,'Catboost_best_params'])['learning_rate'],
                     l2_leaf_reg = ast.literal_eval(ml_results_best_params.loc[share,'Catboost_best_params'])['l2_leaf_reg'],
                     silent = ast.literal_eval(ml_results_best_params.loc[share,'Catboost_best_params'])['silent'])


    model.fit(X_train, y_train)
    
    y_pred_2 = model.predict(X_train.iloc[-1:])
    
    #xgboost
    
    model = xgb.XGBRegressor(n_estimators = ast.literal_eval(ml_results_best_params.loc[share,'XGBoost_best_params'])['n_estimators'],
                     colsample_bytree = ast.literal_eval(ml_results_best_params.loc[share,'XGBoost_best_params'])['colsample_bytree'],
                     learning_rate = ast.literal_eval(ml_results_best_params.loc[share,'XGBoost_best_params'])['learning_rate'],
                     max_depth = ast.literal_eval(ml_results_best_params.loc[share,'XGBoost_best_params'])['max_depth'],
                     min_child_weight = ast.literal_eval(ml_results_best_params.loc[share,'XGBoost_best_params'])['min_child_weight'],
                     random_state = ast.literal_eval(ml_results_best_params.loc[share,'XGBoost_best_params'])['random_state'],
                     reg_alpha = ast.literal_eval(ml_results_best_params.loc[share,'XGBoost_best_params'])['reg_alpha'],
                     reg_lambda = ast.literal_eval(ml_results_best_params.loc[share,'XGBoost_best_params'])['reg_lambda'],
                     subsample = ast.literal_eval(ml_results_best_params.loc[share,'XGBoost_best_params'])['subsample'])


    model.fit(X_train, y_train)
    
    y_pred_3 = model.predict(X_train.iloc[-1:])
    
    
    # rf
    rf = RandomForestRegressor(n_estimators = ast.literal_eval(ml_results_best_params.loc[share,'RF_best_params'])['n_estimators'],
                      max_features=ast.literal_eval(ml_results_best_params.loc[share,'RF_best_params'])['max_features'],
                      min_samples_leaf=ast.literal_eval(ml_results_best_params.loc[share,'RF_best_params'])['min_samples_leaf'],
                      min_samples_split=ast.literal_eval(ml_results_best_params.loc[share,'RF_best_params'])['min_samples_split'],
                      max_depth=ast.literal_eval(ml_results_best_params.loc[share,'RF_best_params'])['max_depth'],
                      bootstrap = ast.literal_eval(ml_results_best_params.loc[share,'RF_best_params'])['bootstrap'])
    rf.fit(X_train, y_train)
    y_pred_4 = rf.predict(X_train.iloc[-1:])
    
    
    #svm
    svm = SVR(C = ast.literal_eval(ml_results_best_params.loc[share,'SVM_best_params'])['C'],
            gamma = ast.literal_eval(ml_results_best_params.loc[share,'SVM_best_params'])['gamma'],
            kernel = ast.literal_eval(ml_results_best_params.loc[share,'SVM_best_params'])['kernel'])

    svm.fit(X_train, y_train)
    y_pred_5 = svm.predict(X_train.iloc[-1:])
    
    #ridge
    ridge = Ridge(alpha=ast.literal_eval(ml_results_best_params.loc[share,'Ridge_best_params'])['alpha'],
                  tol=ast.literal_eval(ml_results_best_params.loc[share,'Ridge_best_params'])['tol'],
                  normalize=ast.literal_eval(ml_results_best_params.loc[share,'Ridge_best_params'])['normalize'])

    ridge.fit(X_train, y_train)
    y_pred_6 = ridge.predict(X_train.iloc[-1:])
    
    #lasso
    lasso = Lasso(alpha = ast.literal_eval(ml_results_best_params.loc[share,'LASSO_best_params'])['alpha'],
                  random_state = ast.literal_eval(ml_results_best_params.loc[share,'LASSO_best_params'])['random_state'],
                  normalize=ast.literal_eval(ml_results_best_params.loc[share,'LASSO_best_params'])['normalize'],
                  tol=ast.literal_eval(ml_results_best_params.loc[share,'LASSO_best_params'])['tol'])

    lasso.fit(X_train, y_train)
    y_pred_7 = lasso.predict(X_train.iloc[-1:])
    
    
    y_pred_avg = (y_pred_1+y_pred_2+y_pred_3+y_pred_4+y_pred_5+y_pred_6+y_pred_7)/7
    
    
    
    # фактическая больше модельной
    if y_train[-1] -  y_pred_avg > gap_10:
        Short_10_all.append(name)
    # модельная больше фактической
    elif y_pred_avg - y_train[-1] > gap_10:
        Long_10_all.append(name)
    elif abs(y_pred_avg - y_train[-1]) < gap_10:
            Hold_all.append(name)
    else:
        continue
        
Recomends_all = pd.DataFrame({'Short':pd.Series(Short_10_rf), 
                                  'Hold':pd.Series(Hold_rf),
                                  'Long':pd.Series(Long_10_rf)})

Recomends_all['Short'] = Recomends_all['Short'].astype(str).str[7:11]
Recomends_all['Hold'] = Recomends_all['Hold'].astype(str).str[7:11]
Recomends_all['Long'] = Recomends_all['Long'].astype(str).str[7:11]
Recomends_all.to_excel('Recomends_data_all.xlsx', sheet_name='recom')        
